In [2]:
import sys
sys.path.insert(0, '..')
import os

import numpy as np
import pandas as pd

label the test data

In [2]:
data_test = pd.read_csv('./data/testData.tsv', sep='\t')
print(data_test.shape)
data_test.head()

(25000, 2)


,id,review
0,12311_10,Naturally in a film who's main themes are of m...
1,8348_2,This movie is a disaster within a disaster fil...
2,5828_4,"All in all, this is a movie for kids. We saw i..."
3,7186_2,Afraid of the Dark left me with the impression...
4,12128_7,A very accurate depiction of small time mob li...


In [3]:
temp = data_test.id.str.split('_')

In [4]:
test_label = []
for i in range(25000):
    test_label.append(1 if int(temp.loc[i][1])>5 else 0)

In [5]:
test_label = pd.DataFrame(test_label)
test_label.columns = ['sentiment']
test_label.head()

,sentiment
0,1
1,0
2,0
3,0
4,1


In [6]:
data_test = data_test.join(test_label)

In [7]:
data_test.head()

,id,review,sentiment
0,12311_10,Naturally in a film who's main themes are of m...,1
1,8348_2,This movie is a disaster within a disaster fil...,0
2,5828_4,"All in all, this is a movie for kids. We saw i...",0
3,7186_2,Afraid of the Dark left me with the impression...,0
4,12128_7,A very accurate depiction of small time mob li...,1


In [8]:
data_test.to_csv('./data/labeledTestData.csv',index=None,encoding='utf8')

In [9]:
labeledTestData = pd.read_csv('./data/labeledTestData.csv')

In [10]:
labeledTestData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
id           25000 non-null object
review       25000 non-null object
sentiment    25000 non-null int64
dtypes: int64(1), object(2)
memory usage: 586.0+ KB


In [11]:
labeledTestData.head()

,id,review,sentiment
0,12311_10,Naturally in a film who's main themes are of m...,1
1,8348_2,This movie is a disaster within a disaster fil...,0
2,5828_4,"All in all, this is a movie for kids. We saw i...",0
3,7186_2,Afraid of the Dark left me with the impression...,0
4,12128_7,A very accurate depiction of small time mob li...,1


preprocess the train and test data

In [12]:
train = pd.read_csv( './data/labeledTrainData.tsv', header=0, delimiter='\t', quoting=3)
test = pd.read_csv('./data/labeledTestData.csv')
unlabeled_train = pd.read_csv( './data/unlabeledTrainData.tsv', header=0, delimiter='\t', quoting=3)

In [13]:
data_all = pd.concat([train,test])
data_all.to_csv('./data/data_all.csv',index=None,encoding='utf8')

In [20]:
data_all = pd.read_csv('./data/data_all.csv', encoding='utf-8')

In [15]:
data_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
id           50000 non-null object
review       50000 non-null object
sentiment    50000 non-null int64
dtypes: int64(1), object(2)
memory usage: 1.1+ MB


In [16]:
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def clean_str(string):
    """
    Tokenization/string cleaning for all datasets 
    Every dataset is lower cased
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)     
    string = re.sub(r"\'s", " \'s", string) 
    string = re.sub(r"\'ve", " \'ve", string) 
    string = re.sub(r"n\'t", " n\'t", string) 
    string = re.sub(r"\'re", " \'re", string) 
    string = re.sub(r"\'d", " \'d", string) 
    string = re.sub(r"\'ll", " \'ll", string) 
    string = re.sub(r",", " , ", string) 
    string = re.sub(r"!", " ! ", string) 
    string = re.sub(r"\(", " \( ", string) 
    string = re.sub(r"\)", " \) ", string) 
    string = re.sub(r"\?", " \? ", string) 
    string = re.sub(r"\s{2,}", " ", string)    
    return string.strip().lower()

stops = set(stopwords.words("english"))
def review_to_words(raw_review):
    review_text = BeautifulSoup(raw_review, "lxml").get_text()  
    words = clean_str(review_text)
    meaningful_words = [w for w in words.split(' ') if not w in stops]
    return(" ".join(meaningful_words))

In [17]:
texts = []
texts_with_stopwords = []
labels = []
for idx in range(data_all.review.shape[0]):
    text = review_to_words(data_all.review[idx])
    texts.append(text)
    texts_with_stopwords.append(clean_str(BeautifulSoup(data_all.review[idx], "lxml").get_text()))
    labels.append(data_all.sentiment[idx])

predict

In [28]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression

text_clf = Pipeline([('vect', TfidfVectorizer(max_df=0.95, min_df=3, ngram_range=(1, 2))),
                     ('clf', LogisticRegression(C=3)),
])

parameters = {'vect__min_df': (3,5),
              'clf__C': (10,15),
}

gs_clf = GridSearchCV(text_clf, parameters, n_jobs=4)
gs_clf = gs_clf.fit(texts_with_stopwords, labels)
print(gs_clf.best_score_)
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

0.91146
clf__C: 15
vect__min_df: 3


In [29]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=3, ngram_range=(1, 2))
tfidf_features_with_stopwords = tfidf_vectorizer.fit_transform(texts_with_stopwords)
lr = LogisticRegression(C=15)
lr.fit(tfidf_features_with_stopwords[:25000],labels[:25000]).score(tfidf_features_with_stopwords[25000:], labels[25000:])

0.90168

In [30]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=3, ngram_range=(1, 2))
tfidf_features_with_stopwords = tfidf_vectorizer.fit_transform(texts_with_stopwords[:25000])
tfidf_features_with_stopwords_test = tfidf_vectorizer.transform(texts_with_stopwords[25000:])
lr = LogisticRegression(C=15)
lr.fit(tfidf_features_with_stopwords,labels[:25000]).score(tfidf_features_with_stopwords_test, labels[25000:])

0.90124

In [33]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=3, ngram_range=(1, 3))
tfidf_features_with_stopwords = tfidf_vectorizer.fit_transform(texts_with_stopwords[:25000])
tfidf_features_with_stopwords_test = tfidf_vectorizer.transform(texts_with_stopwords[25000:])
lr = LogisticRegression(C=15)
lr.fit(tfidf_features_with_stopwords,labels[:25000]).score(tfidf_features_with_stopwords_test, labels[25000:])

0.90328

use character features

In [34]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from scipy.sparse import hstack

tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=3, ngram_range=(1, 2))
tfidf_vectorizer_char = TfidfVectorizer(max_df=0.95, min_df=5, analyzer='char', ngram_range=(1, 4))
tfidf_features_with_stopwords = tfidf_vectorizer.fit_transform(texts_with_stopwords)
tfidf_features_with_stopwords_char = tfidf_vectorizer_char.fit_transform(texts_with_stopwords)

tfidf_matrix_word_char =  hstack((tfidf_features_with_stopwords, tfidf_features_with_stopwords_char))

In [35]:
lr = LogisticRegression(C=10)
lr.fit(tfidf_features_with_stopwords[:25000],labels[:25000]).score(tfidf_features_with_stopwords[25000:], labels[25000:])

0.9018

train word2vec

In [20]:
data_with_unlabeled_train = pd.concat([train,test,unlabeled_train]).fillna(0)
data_with_unlabeled_train.to_csv('./data/data_with_unlabeled_train.csv',index=None,encoding='utf8')

In [3]:
data_with_unlabeled_train = pd.read_csv('./data/data_with_unlabeled_train.csv', encoding='utf-8')

In [22]:
data_with_unlabeled_train.head(2)

,id,review,sentiment
0,"""5814_8""","""With all this stuff going down at the moment ...",1.0
1,"""2381_9""","""\""The Classic War of the Worlds\"" by Timothy ...",1.0


In [23]:
data_with_unlabeled_train.tail(2)

,id,review,sentiment
99998,"""40155_0""","""Not a bad story, but the low budget rears its...",0.0
99999,"""35270_0""","""This not-very-good mummy-alien flick does fea...",0.0


In [24]:
data_with_unlabeled_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 3 columns):
id           100000 non-null object
review       100000 non-null object
sentiment    100000 non-null float64
dtypes: float64(1), object(2)
memory usage: 2.3+ MB


In [ ]:
texts_with_unlabeled_train = []
for idx in range(data_with_unlabeled_train.review.shape[0]):
    texts_with_stopwords.append(clean_str(BeautifulSoup(data_with_unlabeled_train.review[idx], "lxml").get_text()))

In [31]:
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
import nltk.data
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
stop_words = set(stopwords.words("english"))

def clean_str(string):
        """
        Tokenization/string cleaning for all datasets 
        Every dataset is lower cased
        """
        string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)     
        string = re.sub(r"\'s", " \'s", string) 
        string = re.sub(r"\'ve", " \'ve", string) 
        string = re.sub(r"n\'t", " n\'t", string) 
        string = re.sub(r"\'re", " \'re", string) 
        string = re.sub(r"\'d", " \'d", string) 
        string = re.sub(r"\'ll", " \'ll", string) 
        string = re.sub(r",", " , ", string) 
        string = re.sub(r"!", " ! ", string) 
        string = re.sub(r"\(", " \( ", string) 
        string = re.sub(r"\)", " \) ", string) 
        string = re.sub(r"\?", " \? ", string) 
        string = re.sub(r"\s{2,}", " ", string)    
        return string.strip().lower().split()

def review_to_words(raw_review, remove_stopwords=False):
        review_text = BeautifulSoup(raw_review, "lxml").get_text()  
        words = clean_str(review_text)
        if remove_stopwords:
            words = [w for w in words if not w in stop_words]
        return words
    
def review_to_sentences(review, remove_stopwords=False):
        raw_sentences = tokenizer.tokenize(review.strip())
        sentences = []
        for raw_sentence in raw_sentences:
            if len(raw_sentence) > 0:
                sentences.append(review_to_words(raw_sentence, remove_stopwords))
        return sentences


class MySentences(object):
    def __init__(self, myfile):
        self.myfile = myfile
        
    def __iter__(self):
        for review in self.myfile:
            raw_sentences = tokenizer.tokenize(review.strip())
            for raw_sentence in raw_sentences:
                if len(raw_sentence) > 0:
                    yield review_to_words(raw_sentence, remove_stopwords=False)

In [32]:
sentences = MySentences(data_with_unlabeled_train.review)  # a memory-friendly iterator

In [6]:
for sen1 in sentences:
    print(sen1)
    break

['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'mj', 'i', "'ve", 'started', 'listening', 'to', 'his', 'music', ',', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', ',', 'watched', 'the', 'wiz', 'and', 'watched', 'moonwalker', 'again']


In [7]:
i=0
for sen1 in sentences:
    i+=1
    print(sen1)
    if i > 10:
        break

['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'mj', 'i', "'ve", 'started', 'listening', 'to', 'his', 'music', ',', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', ',', 'watched', 'the', 'wiz', 'and', 'watched', 'moonwalker', 'again']
['maybe', 'i', 'just', 'want', 'to', 'get', 'a', 'certain', 'insight', 'into', 'this', 'guy', 'who', 'i', 'thought', 'was', 'really', 'cool', 'in', 'the', 'eighties', 'just', 'to', 'maybe', 'make', 'up', 'my', 'mind', 'whether', 'he', 'is', 'guilty', 'or', 'innocent']
['moonwalker', 'is', 'part', 'biography', ',', 'part', 'feature', 'film', 'which', 'i', 'remember', 'going', 'to', 'see', 'at', 'the', 'cinema', 'when', 'it', 'was', 'originally', 'released']
['some', 'of', 'it', 'has', 'subtle', 'messages', 'about', 'mj', "'s", 'feeling', 'towards', 'the', 'press', 'and', 'also', 'the', 'obvious', 'message', 'of', 'drugs', 'are', 'bad', "m'kay", 'visually', 'impressive', 'but', 'of', 'course', 'this', 'is',

In [43]:
sentences = []  # Initialize an empty list of sentences

for review in data_with_unlabeled_train["review"]:
    sentences += review_to_sentences(review)

C:\Users\sun\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\Users\sun\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'...'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\Users\sun\Anaconda3\lib\site-packages\bs4\__init__.py:282: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\sun\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'..'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful S

In [44]:
print(len(sentences))
print(sentences[0])

1057275
['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'mj', 'i', "'ve", 'started', 'listening', 'to', 'his', 'music', ',', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', ',', 'watched', 'the', 'wiz', 'and', 'watched', 'moonwalker', 'again']


In [33]:
from gensim.models.fasttext import FastText
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

num_features = 300    # Word vector dimensionality                      
min_word_count = 5   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words
sg = 1
hs = 0

print("Training model...")
model = FastText(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling, sg = sg)

model_name = "300features_40minwords_10context_fasttext"
model.save(model_name)

2018-10-19 19:13:00,159 : INFO : collecting all words and their counts
2018-10-19 19:13:00,163 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


Training model...


C:\Users\sun\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
2018-10-19 19:13:02,690 : INFO : PROGRESS: at sentence #10000, processed 241944 words, keeping 18449 word types
C:\Users\sun\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'...'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\Users\sun\Anaconda3\lib\site-packages\bs4\__init__.py:282: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
2018-10-19 19:13:05,277 : INFO : PROGRESS: at sentence #20000, processed 483720 w

2018-10-19 19:15:18,570 : INFO : PROGRESS: at sentence #560000, processed 13296240 words, keeping 118743 word types
2018-10-19 19:15:21,043 : INFO : PROGRESS: at sentence #570000, processed 13536737 words, keeping 119869 word types
2018-10-19 19:15:23,511 : INFO : PROGRESS: at sentence #580000, processed 13776414 words, keeping 120994 word types
2018-10-19 19:15:25,972 : INFO : PROGRESS: at sentence #590000, processed 14013310 words, keeping 122095 word types
2018-10-19 19:15:28,440 : INFO : PROGRESS: at sentence #600000, processed 14259336 words, keeping 123261 word types
C:\Users\sun\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'... ...'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
2018-10-19 19:15:30,899 : INFO : PROGRESS: at sentence #610000, processed 14499025 words, keeping 124344 word types
2018-10-19 19:15:33,350 : INFO : PROGRESS: at sentence #620000, proces

2018-10-19 19:17:25,354 : INFO : estimated required memory for 55895 words, 244023 buckets and 300 dimensions: 467819764 bytes
2018-10-19 19:17:25,363 : INFO : resetting layer weights
2018-10-19 19:17:27,694 : INFO : Total number of ngrams is 244023
2018-10-19 19:17:30,234 : INFO : training model with 4 workers on 55895 vocabulary and 300 features, using sg=1 hs=0 sample=0.001 negative=5 window=10
2018-10-19 19:17:31,292 : INFO : EPOCH 1 - PROGRESS: at 0.28% examples, 48519 words/s, in_qsize 0, out_qsize 0
2018-10-19 19:17:32,298 : INFO : EPOCH 1 - PROGRESS: at 0.59% examples, 52956 words/s, in_qsize 0, out_qsize 0
2018-10-19 19:17:33,303 : INFO : EPOCH 1 - PROGRESS: at 0.90% examples, 54583 words/s, in_qsize 0, out_qsize 0
2018-10-19 19:17:34,318 : INFO : EPOCH 1 - PROGRESS: at 1.21% examples, 55195 words/s, in_qsize 0, out_qsize 0
2018-10-19 19:17:35,338 : INFO : EPOCH 1 - PROGRESS: at 1.49% examples, 54052 words/s, in_qsize 0, out_qsize 0
2018-10-19 19:17:36,458 : INFO : EPOCH 1 - P

...

2018-10-19 19:38:57,959 : INFO : EPOCH 4 - PROGRESS: at 96.20% examples, 56401 words/s, in_qsize 0, out_qsize 0
2018-10-19 19:38:59,023 : INFO : EPOCH 4 - PROGRESS: at 96.53% examples, 56394 words/s, in_qsize 0, out_qsize 0
2018-10-19 19:39:00,057 : INFO : EPOCH 4 - PROGRESS: at 96.85% examples, 56394 words/s, in_qsize 0, out_qsize 0
2018-10-19 19:39:01,187 : INFO : EPOCH 4 - PROGRESS: at 97.19% examples, 56398 words/s, in_qsize 0, out_qsize 0
2018-10-19 19:39:02,206 : INFO : EPOCH 4 - PROGRESS: at 97.51% examples, 56400 words/s, in_qsize 0, out_qsize 0
2018-10-19 19:39:03,332 : INFO : EPOCH 4 - PROGRESS: at 97.85% examples, 56405 words/s, in_qsize 0, out_qsize 0
2018-10-19 19:39:04,341 : INFO : EPOCH 4 - PROGRESS: at 98.16% examples, 56408 words/s, in_qsize 0, out_qsize 0
2018-10-19 19:39:05,460 : INFO : EPOCH 4 - PROGRESS: at 98.50% examples, 56415 words/s, in_qsize 0, out_qsize 0
2018-10-19 19:39:06,489 : INFO : EPOCH 4 - PROGRESS: at 98.82% examples, 56415 words/s, in_qsize 0, out_

2018-10-19 19:40:10,799 : INFO : EPOCH 5 - PROGRESS: at 18.56% examples, 56102 words/s, in_qsize 0, out_qsize 0
2018-10-19 19:40:11,918 : INFO : EPOCH 5 - PROGRESS: at 18.91% examples, 56139 words/s, in_qsize 0, out_qsize 0
2018-10-19 19:40:12,964 : INFO : EPOCH 5 - PROGRESS: at 19.23% examples, 56127 words/s, in_qsize 0, out_qsize 0
2018-10-19 19:40:14,011 : INFO : EPOCH 5 - PROGRESS: at 19.55% examples, 56120 words/s, in_qsize 0, out_qsize 0
2018-10-19 19:40:15,021 : INFO : EPOCH 5 - PROGRESS: at 19.86% examples, 56139 words/s, in_qsize 0, out_qsize 0
2018-10-19 19:40:16,050 : INFO : EPOCH 5 - PROGRESS: at 20.18% examples, 56145 words/s, in_qsize 0, out_qsize 0
2018-10-19 19:40:17,077 : INFO : EPOCH 5 - PROGRESS: at 20.50% examples, 56149 words/s, in_qsize 0, out_qsize 0
2018-10-19 19:40:18,083 : INFO : EPOCH 5 - PROGRESS: at 20.81% examples, 56171 words/s, in_qsize 0, out_qsize 0
2018-10-19 19:40:19,104 : INFO : EPOCH 5 - PROGRESS: at 21.12% examples, 56181 words/s, in_qsize 0, out_

2018-10-19 19:41:27,506 : INFO : EPOCH 5 - PROGRESS: at 42.19% examples, 56048 words/s, in_qsize 0, out_qsize 0
2018-10-19 19:41:28,574 : INFO : EPOCH 5 - PROGRESS: at 42.53% examples, 56034 words/s, in_qsize 0, out_qsize 0
2018-10-19 19:41:29,597 : INFO : EPOCH 5 - PROGRESS: at 42.84% examples, 56037 words/s, in_qsize 0, out_qsize 0
2018-10-19 19:41:30,606 : INFO : EPOCH 5 - PROGRESS: at 43.16% examples, 56048 words/s, in_qsize 0, out_qsize 0
2018-10-19 19:41:31,656 : INFO : EPOCH 5 - PROGRESS: at 43.48% examples, 56042 words/s, in_qsize 0, out_qsize 0
2018-10-19 19:41:32,670 : INFO : EPOCH 5 - PROGRESS: at 43.80% examples, 56049 words/s, in_qsize 0, out_qsize 0
2018-10-19 19:41:33,700 : INFO : EPOCH 5 - PROGRESS: at 44.11% examples, 56050 words/s, in_qsize 0, out_qsize 0
2018-10-19 19:41:34,733 : INFO : EPOCH 5 - PROGRESS: at 44.44% examples, 56050 words/s, in_qsize 0, out_qsize 0
2018-10-19 19:41:35,775 : INFO : EPOCH 5 - PROGRESS: at 44.76% examples, 56046 words/s, in_qsize 0, out_

2018-10-19 19:42:43,997 : INFO : EPOCH 5 - PROGRESS: at 65.67% examples, 56172 words/s, in_qsize 0, out_qsize 0
2018-10-19 19:42:45,015 : INFO : EPOCH 5 - PROGRESS: at 65.98% examples, 56175 words/s, in_qsize 0, out_qsize 0
2018-10-19 19:42:46,060 : INFO : EPOCH 5 - PROGRESS: at 66.31% examples, 56172 words/s, in_qsize 0, out_qsize 0
2018-10-19 19:42:47,074 : INFO : EPOCH 5 - PROGRESS: at 66.62% examples, 56177 words/s, in_qsize 0, out_qsize 0
2018-10-19 19:42:48,090 : INFO : EPOCH 5 - PROGRESS: at 66.94% examples, 56181 words/s, in_qsize 0, out_qsize 0
2018-10-19 19:42:49,099 : INFO : EPOCH 5 - PROGRESS: at 67.24% examples, 56186 words/s, in_qsize 0, out_qsize 0
2018-10-19 19:42:50,117 : INFO : EPOCH 5 - PROGRESS: at 67.56% examples, 56191 words/s, in_qsize 0, out_qsize 0
2018-10-19 19:42:51,132 : INFO : EPOCH 5 - PROGRESS: at 67.86% examples, 56195 words/s, in_qsize 0, out_qsize 0
2018-10-19 19:42:52,222 : INFO : EPOCH 5 - PROGRESS: at 68.19% examples, 56212 words/s, in_qsize 0, out_

2018-10-19 19:44:00,389 : INFO : EPOCH 5 - PROGRESS: at 89.09% examples, 56281 words/s, in_qsize 0, out_qsize 0
2018-10-19 19:44:01,398 : INFO : EPOCH 5 - PROGRESS: at 89.41% examples, 56285 words/s, in_qsize 0, out_qsize 0
2018-10-19 19:44:02,438 : INFO : EPOCH 5 - PROGRESS: at 89.73% examples, 56283 words/s, in_qsize 0, out_qsize 0
2018-10-19 19:44:03,479 : INFO : EPOCH 5 - PROGRESS: at 90.05% examples, 56281 words/s, in_qsize 0, out_qsize 0
2018-10-19 19:44:04,482 : INFO : EPOCH 5 - PROGRESS: at 90.36% examples, 56285 words/s, in_qsize 0, out_qsize 0
2018-10-19 19:44:05,505 : INFO : EPOCH 5 - PROGRESS: at 90.67% examples, 56289 words/s, in_qsize 0, out_qsize 0
2018-10-19 19:44:06,628 : INFO : EPOCH 5 - PROGRESS: at 91.01% examples, 56295 words/s, in_qsize 0, out_qsize 0
2018-10-19 19:44:07,636 : INFO : EPOCH 5 - PROGRESS: at 91.32% examples, 56300 words/s, in_qsize 0, out_qsize 0
2018-10-19 19:44:08,675 : INFO : EPOCH 5 - PROGRESS: at 91.65% examples, 56297 words/s, in_qsize 0, out_

In [8]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

num_features = 300    # Word vector dimensionality                      
min_word_count = 5   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words
sg = 1
hs = 0


from gensim.models import word2vec
print("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling, sg = sg)

model_name = "300features_40minwords_10context_sg"
model.save(model_name)

C:\Users\sun\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
2018-10-19 17:47:08,140 : INFO : 'pattern' package not found; tag filters are not available for English
2018-10-19 17:47:08,144 : INFO : collecting all words and their counts
2018-10-19 17:47:08,155 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


Training model...


C:\Users\sun\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
2018-10-19 17:47:10,691 : INFO : PROGRESS: at sentence #10000, processed 241944 words, keeping 18449 word types
C:\Users\sun\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'...'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\Users\sun\Anaconda3\lib\site-packages\bs4\__init__.py:282: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
2018-10-19 17:47:13,243 : INFO : PROGRESS: at sentence #20000, processed 483720 w

2018-10-19 17:49:27,017 : INFO : PROGRESS: at sentence #560000, processed 13296240 words, keeping 118743 word types
2018-10-19 17:49:29,467 : INFO : PROGRESS: at sentence #570000, processed 13536737 words, keeping 119869 word types
2018-10-19 17:49:31,919 : INFO : PROGRESS: at sentence #580000, processed 13776414 words, keeping 120994 word types
2018-10-19 17:49:34,403 : INFO : PROGRESS: at sentence #590000, processed 14013310 words, keeping 122095 word types
2018-10-19 17:49:36,892 : INFO : PROGRESS: at sentence #600000, processed 14259336 words, keeping 123261 word types
C:\Users\sun\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'... ...'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
2018-10-19 17:49:39,364 : INFO : PROGRESS: at sentence #610000, processed 14499025 words, keeping 124344 word types
2018-10-19 17:49:41,812 : INFO : PROGRESS: at sentence #620000, proces

2018-10-19 17:51:29,567 : INFO : estimated required memory for 55895 words and 300 dimensions: 162095500 bytes
2018-10-19 17:51:29,567 : INFO : resetting layer weights
2018-10-19 17:51:30,203 : INFO : training model with 4 workers on 55895 vocabulary and 300 features, using sg=1 hs=0 sample=0.001 negative=5 window=10
2018-10-19 17:51:31,258 : INFO : EPOCH 1 - PROGRESS: at 0.35% examples, 62409 words/s, in_qsize 0, out_qsize 0
2018-10-19 17:51:32,330 : INFO : EPOCH 1 - PROGRESS: at 0.74% examples, 65092 words/s, in_qsize 0, out_qsize 0
2018-10-19 17:51:33,406 : INFO : EPOCH 1 - PROGRESS: at 1.14% examples, 65887 words/s, in_qsize 0, out_qsize 0
2018-10-19 17:51:34,488 : INFO : EPOCH 1 - PROGRESS: at 1.53% examples, 66090 words/s, in_qsize 0, out_qsize 0
2018-10-19 17:51:35,545 : INFO : EPOCH 1 - PROGRESS: at 1.92% examples, 66662 words/s, in_qsize 0, out_qsize 0
2018-10-19 17:51:36,643 : INFO : EPOCH 1 - PROGRESS: at 2.32% examples, 66524 words/s, in_qsize 0, out_qsize 0
2018-10-19 17:5

...

2018-10-19 18:08:21,425 : INFO : EPOCH 4 - PROGRESS: at 65.79% examples, 65505 words/s, in_qsize 0, out_qsize 0
2018-10-19 18:08:22,434 : INFO : EPOCH 4 - PROGRESS: at 66.15% examples, 65499 words/s, in_qsize 0, out_qsize 0
2018-10-19 18:08:23,438 : INFO : EPOCH 4 - PROGRESS: at 66.50% examples, 65498 words/s, in_qsize 0, out_qsize 0
2018-10-19 18:08:24,441 : INFO : EPOCH 4 - PROGRESS: at 66.85% examples, 65496 words/s, in_qsize 0, out_qsize 0
2018-10-19 18:08:25,542 : INFO : EPOCH 4 - PROGRESS: at 67.24% examples, 65498 words/s, in_qsize 0, out_qsize 0
2018-10-19 18:08:26,557 : INFO : EPOCH 4 - PROGRESS: at 67.59% examples, 65492 words/s, in_qsize 0, out_qsize 0
2018-10-19 18:08:27,594 : INFO : EPOCH 4 - PROGRESS: at 67.97% examples, 65516 words/s, in_qsize 0, out_qsize 0
2018-10-19 18:08:28,673 : INFO : EPOCH 4 - PROGRESS: at 68.35% examples, 65525 words/s, in_qsize 1, out_qsize 0
2018-10-19 18:08:29,692 : INFO : EPOCH 4 - PROGRESS: at 68.70% examples, 65518 words/s, in_qsize 0, out_

2018-10-19 18:09:39,762 : INFO : EPOCH 4 - PROGRESS: at 93.65% examples, 65555 words/s, in_qsize 0, out_qsize 0
2018-10-19 18:09:40,821 : INFO : EPOCH 4 - PROGRESS: at 94.04% examples, 65566 words/s, in_qsize 0, out_qsize 0
2018-10-19 18:09:41,827 : INFO : EPOCH 4 - PROGRESS: at 94.40% examples, 65562 words/s, in_qsize 0, out_qsize 0
2018-10-19 18:09:42,906 : INFO : EPOCH 4 - PROGRESS: at 94.80% examples, 65569 words/s, in_qsize 0, out_qsize 0
2018-10-19 18:09:43,998 : INFO : EPOCH 4 - PROGRESS: at 95.20% examples, 65573 words/s, in_qsize 0, out_qsize 0
2018-10-19 18:09:45,007 : INFO : EPOCH 4 - PROGRESS: at 95.56% examples, 65571 words/s, in_qsize 0, out_qsize 0
2018-10-19 18:09:46,080 : INFO : EPOCH 4 - PROGRESS: at 95.95% examples, 65578 words/s, in_qsize 0, out_qsize 0
2018-10-19 18:09:47,184 : INFO : EPOCH 4 - PROGRESS: at 96.36% examples, 65578 words/s, in_qsize 0, out_qsize 0
2018-10-19 18:09:48,239 : INFO : EPOCH 4 - PROGRESS: at 96.73% examples, 65564 words/s, in_qsize 0, out_

2018-10-19 18:10:54,552 : INFO : EPOCH 5 - PROGRESS: at 20.73% examples, 66555 words/s, in_qsize 0, out_qsize 0
2018-10-19 18:10:55,634 : INFO : EPOCH 5 - PROGRESS: at 21.12% examples, 66563 words/s, in_qsize 0, out_qsize 0
2018-10-19 18:10:56,711 : INFO : EPOCH 5 - PROGRESS: at 21.52% examples, 66574 words/s, in_qsize 0, out_qsize 0
2018-10-19 18:10:57,724 : INFO : EPOCH 5 - PROGRESS: at 21.87% examples, 66539 words/s, in_qsize 0, out_qsize 0
2018-10-19 18:10:58,774 : INFO : EPOCH 5 - PROGRESS: at 22.25% examples, 66578 words/s, in_qsize 0, out_qsize 0
2018-10-19 18:10:59,867 : INFO : EPOCH 5 - PROGRESS: at 22.65% examples, 66575 words/s, in_qsize 0, out_qsize 0
2018-10-19 18:11:00,869 : INFO : EPOCH 5 - PROGRESS: at 23.02% examples, 66552 words/s, in_qsize 0, out_qsize 0
2018-10-19 18:11:01,886 : INFO : EPOCH 5 - PROGRESS: at 23.39% examples, 66512 words/s, in_qsize 0, out_qsize 0
2018-10-19 18:11:02,901 : INFO : EPOCH 5 - PROGRESS: at 23.77% examples, 66478 words/s, in_qsize 0, out_

2018-10-19 18:12:13,389 : INFO : EPOCH 5 - PROGRESS: at 49.52% examples, 66376 words/s, in_qsize 0, out_qsize 0
2018-10-19 18:12:14,440 : INFO : EPOCH 5 - PROGRESS: at 49.90% examples, 66394 words/s, in_qsize 0, out_qsize 0
2018-10-19 18:12:15,488 : INFO : EPOCH 5 - PROGRESS: at 50.28% examples, 66415 words/s, in_qsize 0, out_qsize 0
2018-10-19 18:12:16,558 : INFO : EPOCH 5 - PROGRESS: at 50.67% examples, 66425 words/s, in_qsize 0, out_qsize 0
2018-10-19 18:12:17,573 : INFO : EPOCH 5 - PROGRESS: at 51.03% examples, 66409 words/s, in_qsize 0, out_qsize 0
2018-10-19 18:12:18,652 : INFO : EPOCH 5 - PROGRESS: at 51.42% examples, 66416 words/s, in_qsize 0, out_qsize 0
2018-10-19 18:12:19,736 : INFO : EPOCH 5 - PROGRESS: at 51.82% examples, 66421 words/s, in_qsize 0, out_qsize 0
2018-10-19 18:12:20,815 : INFO : EPOCH 5 - PROGRESS: at 52.21% examples, 66427 words/s, in_qsize 0, out_qsize 0
2018-10-19 18:12:21,888 : INFO : EPOCH 5 - PROGRESS: at 52.60% examples, 66435 words/s, in_qsize 0, out_

2018-10-19 18:13:32,399 : INFO : EPOCH 5 - PROGRESS: at 78.18% examples, 66644 words/s, in_qsize 0, out_qsize 0
2018-10-19 18:13:33,462 : INFO : EPOCH 5 - PROGRESS: at 78.57% examples, 66652 words/s, in_qsize 0, out_qsize 0
2018-10-19 18:13:34,524 : INFO : EPOCH 5 - PROGRESS: at 78.95% examples, 66659 words/s, in_qsize 0, out_qsize 0
2018-10-19 18:13:35,602 : INFO : EPOCH 5 - PROGRESS: at 79.35% examples, 66662 words/s, in_qsize 0, out_qsize 0
2018-10-19 18:13:36,688 : INFO : EPOCH 5 - PROGRESS: at 79.75% examples, 66664 words/s, in_qsize 0, out_qsize 0
2018-10-19 18:13:37,764 : INFO : EPOCH 5 - PROGRESS: at 80.14% examples, 66668 words/s, in_qsize 0, out_qsize 0
2018-10-19 18:13:38,868 : INFO : EPOCH 5 - PROGRESS: at 80.53% examples, 66661 words/s, in_qsize 0, out_qsize 0
2018-10-19 18:13:39,950 : INFO : EPOCH 5 - PROGRESS: at 80.93% examples, 66663 words/s, in_qsize 0, out_qsize 0
2018-10-19 18:13:41,032 : INFO : EPOCH 5 - PROGRESS: at 81.32% examples, 66665 words/s, in_qsize 0, out_

In [45]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

num_features = 300    # Word vector dimensionality                      
min_word_count = 5   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words


from gensim.models import word2vec
print("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

model_name = "300features_40minwords_10context"
model.save(model_name)

2018-10-19 16:13:39,199 : INFO : collecting all words and their counts
2018-10-19 16:13:39,200 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-10-19 16:13:39,237 : INFO : PROGRESS: at sentence #10000, processed 241944 words, keeping 18449 word types
2018-10-19 16:13:39,278 : INFO : PROGRESS: at sentence #20000, processed 483720 words, keeping 26155 word types
2018-10-19 16:13:39,314 : INFO : PROGRESS: at sentence #30000, processed 718909 words, keeping 31608 word types
2018-10-19 16:13:39,352 : INFO : PROGRESS: at sentence #40000, processed 961259 words, keeping 36357 word types
2018-10-19 16:13:39,388 : INFO : PROGRESS: at sentence #50000, processed 1196249 words, keeping 40156 word types


Training model...


2018-10-19 16:13:39,424 : INFO : PROGRESS: at sentence #60000, processed 1433089 words, keeping 43426 word types
2018-10-19 16:13:39,465 : INFO : PROGRESS: at sentence #70000, processed 1671705 words, keeping 46385 word types
2018-10-19 16:13:39,499 : INFO : PROGRESS: at sentence #80000, processed 1906221 words, keeping 49064 word types
2018-10-19 16:13:39,536 : INFO : PROGRESS: at sentence #90000, processed 2146491 words, keeping 51812 word types
2018-10-19 16:13:39,573 : INFO : PROGRESS: at sentence #100000, processed 2384047 words, keeping 54189 word types
2018-10-19 16:13:39,607 : INFO : PROGRESS: at sentence #110000, processed 2619479 words, keeping 56320 word types
2018-10-19 16:13:39,646 : INFO : PROGRESS: at sentence #120000, processed 2857001 words, keeping 58637 word types
2018-10-19 16:13:39,684 : INFO : PROGRESS: at sentence #130000, processed 3099128 words, keeping 60668 word types
2018-10-19 16:13:39,717 : INFO : PROGRESS: at sentence #140000, processed 3326972 words, kee

2018-10-19 16:13:42,224 : INFO : PROGRESS: at sentence #780000, processed 18592424 words, keeping 140858 word types
2018-10-19 16:13:42,261 : INFO : PROGRESS: at sentence #790000, processed 18834700 words, keeping 141726 word types
2018-10-19 16:13:42,298 : INFO : PROGRESS: at sentence #800000, processed 19071544 words, keeping 142662 word types
2018-10-19 16:13:42,339 : INFO : PROGRESS: at sentence #810000, processed 19310448 words, keeping 143576 word types
2018-10-19 16:13:42,375 : INFO : PROGRESS: at sentence #820000, processed 19550803 words, keeping 144444 word types
2018-10-19 16:13:42,417 : INFO : PROGRESS: at sentence #830000, processed 19793367 words, keeping 145258 word types
2018-10-19 16:13:42,456 : INFO : PROGRESS: at sentence #840000, processed 20034333 words, keeping 146131 word types
2018-10-19 16:13:42,496 : INFO : PROGRESS: at sentence #850000, processed 20272773 words, keeping 146980 word types
2018-10-19 16:13:42,537 : INFO : PROGRESS: at sentence #860000, processe

2018-10-19 16:14:13,277 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-10-19 16:14:13,278 : INFO : EPOCH - 2 : training on 25242693 raw words (18336279 effective words) took 14.4s, 1273841 effective words/s
2018-10-19 16:14:14,290 : INFO : EPOCH 3 - PROGRESS: at 6.68% examples, 1220382 words/s, in_qsize 7, out_qsize 0
2018-10-19 16:14:15,296 : INFO : EPOCH 3 - PROGRESS: at 13.61% examples, 1237482 words/s, in_qsize 7, out_qsize 0
2018-10-19 16:14:16,300 : INFO : EPOCH 3 - PROGRESS: at 20.34% examples, 1233864 words/s, in_qsize 7, out_qsize 0
2018-10-19 16:14:17,301 : INFO : EPOCH 3 - PROGRESS: at 27.14% examples, 1234701 words/s, in_qsize 7, out_qsize 0
2018-10-19 16:14:18,309 : INFO : EPOCH 3 - PROGRESS: at 34.05% examples, 1236634 words/s, in_qsize 7, out_qsize 0
2018-10-19 16:14:19,309 : INFO : EPOCH 3 - PROGRESS: at 41.08% examples, 1243012 words/s, in_qsize 7, out_qsize 0
2018-10-19 16:14:20,312 : INFO : EPOCH 3 - PROGRESS: at 47.88% examples, 1240755 word

In [9]:
model.wv.doesnt_match("man woman child kitchen".split())

2018-10-19 18:20:12,075 : INFO : precomputing L2-norms of word weight vectors


'kitchen'

In [10]:
model.wv.doesnt_match("france england germany berlin".split())

'berlin'

In [11]:
model.wv.doesnt_match("paris berlin london austria".split())

'london'

In [12]:
model.wv.most_similar("man")

[('woman', 0.5533495545387268),
 ('guy', 0.5364952683448792),
 ('men', 0.5345921516418457),
 ('kashmiri', 0.5059187412261963),
 ('chap', 0.4919845461845398),
 ('objector', 0.4828798770904541),
 ('romono', 0.48286956548690796),
 ('mans', 0.4813400208950043),
 ('crim', 0.47416964173316956),
 ('lagosi', 0.4738166928291321)]

In [13]:
model.wv.most_similar("queen")

[('latifah', 0.5422337055206299),
 ('ellery', 0.5357602834701538),
 ('viveca', 0.4887978136539459),
 ('sandahl', 0.4886631965637207),
 ('sheeba', 0.4884185791015625),
 ('amidala', 0.4856964945793152),
 ('queens', 0.48544973134994507),
 ('lindfors', 0.48037341237068176),
 ('gedren', 0.4802626073360443),
 ('king', 0.47563061118125916)]

In [14]:
model.wv.most_similar("awful")

[('terrible', 0.7587670683860779),
 ('horrible', 0.7220320105552673),
 ('atrocious', 0.6966452598571777),
 ('dreadful', 0.6448382139205933),
 ('appalling', 0.6320473551750183),
 ('abysmal', 0.6299008727073669),
 ('horrendous', 0.629846453666687),
 ('apalling', 0.6284140348434448),
 ('horrid', 0.6059893369674683),
 ('bad', 0.6028908491134644)]

In [15]:
model.wv.most_similar("good")

[('great', 0.6640492677688599),
 ('decent', 0.6411788463592529),
 ('bad', 0.5928826332092285),
 ("'great'", 0.5730583667755127),
 ('nice', 0.5669047832489014),
 ('actualy', 0.5550702810287476),
 ('allright', 0.549493670463562),
 ('dissapointing', 0.5472931861877441),
 ('definate', 0.5460936427116394),
 ('awsome', 0.5422983169555664)]

In [16]:
model.wv.most_similar("better")

[('worse', 0.651874303817749),
 ('edgier', 0.583482027053833),
 ('creepier', 0.5491827130317688),
 ('funnier', 0.5470982193946838),
 ('pornos', 0.5457853078842163),
 ('cb4', 0.5454648733139038),
 ('worser', 0.5402961373329163),
 ('classier', 0.5398609638214111),
 ('snappier', 0.539228618144989),
 ('salvageable', 0.530937910079956)]

In [17]:
model.wv.most_similar("apple")

[('pretzels', 0.5084955096244812),
 ('pecan', 0.501787543296814),
 ('cider', 0.4990856647491455),
 ('buttered', 0.49770110845565796),
 ('bop', 0.4866560995578766),
 ('macs', 0.48226624727249146),
 ('buzzard', 0.47121721506118774),
 ('baking', 0.4697813391685486),
 ('sausage', 0.4682459235191345),
 ('spoonful', 0.46704697608947754)]

In [18]:
model.wv.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)

[('queen', 0.47335630655288696)]

In [19]:
model.wv.similarity('woman', 'man')

0.5533495673999537

In [66]:
word_vectors = model.wv
del model

In [80]:
word_vectors.get_keras_embedding??

In [81]:
data_all.tail(2)

,id,review,sentiment
49998,7772_8,"From the opening sequence, filled with black a...",1
49999,11465_10,This is a great horror film for people who don...,1


In [21]:
from bs4 import BeautifulSoup
import re

def clean_str(string):
    """
    Tokenization/string cleaning for all datasets 
    Every dataset is lower cased
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)     
    string = re.sub(r"\'s", " \'s", string) 
    string = re.sub(r"\'ve", " \'ve", string) 
    string = re.sub(r"n\'t", " n\'t", string) 
    string = re.sub(r"\'re", " \'re", string) 
    string = re.sub(r"\'d", " \'d", string) 
    string = re.sub(r"\'ll", " \'ll", string) 
    string = re.sub(r",", " , ", string) 
    string = re.sub(r"!", " ! ", string) 
    string = re.sub(r"\(", " \( ", string) 
    string = re.sub(r"\)", " \) ", string) 
    string = re.sub(r"\?", " \? ", string) 
    string = re.sub(r"\s{2,}", " ", string)    
    return string.strip().lower()

texts = []
labels = []
for idx in range(data_all.review.shape[0]):
    text = BeautifulSoup(data_all.review[idx], "lxml")
    texts.append(clean_str(text.get_text()))
    labels.append(data_all.sentiment[idx])

In [22]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten, concatenate, TimeDistributed
from keras.layers import Conv1D, MaxPooling1D, Dropout, Concatenate, SpatialDropout1D, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.layers import Bidirectional, LSTM, GRU, CuDNNLSTM, CuDNNGRU
from keras.models import Model

from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers

MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.2

C:\Users\sun\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [23]:
tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

C:\Users\sun\Anaconda3\lib\site-packages\keras_preprocessing\text.py:175: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


Found 114066 unique tokens.


In [24]:
len(sequences)

50000

In [25]:
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (50000, 1000)
Shape of label tensor: (50000, 2)


In [26]:
'''indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])'''

x_train = data[:25000]
y_train = labels[:25000]
x_val = data[25000:]
y_val = labels[25000:]

print('Number of positive and negative reviews in traing and validation set ')
print(y_train.sum(axis=0))
print(y_val.sum(axis=0))

Number of positive and negative reviews in traing and validation set 
[12500. 12500.]
[12500. 12500.]


In [34]:
EMBEDDING_DIM = 300
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
j = 0
for word, i in word_index.items():
    try:
        embedding_vector = model.wv.get_vector(word)
    except:
        j += 1
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
print(j)

64


In [35]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

In [36]:
# use  fasttext
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_dropout = SpatialDropout1D(0.3)(embedded_sequences)
l_gru = Bidirectional(CuDNNGRU(100, return_sequences=True))(l_dropout)
l_cov = Conv1D(64, kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(l_gru)
avg_pool = GlobalAveragePooling1D()(l_cov)
max_pool = GlobalMaxPooling1D()(l_cov)
l_conc = concatenate([avg_pool, max_pool])
preds = Dense(2, activation='softmax')(l_conc)
model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

print("model fitting - RCNN")
model.summary()
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          nb_epoch=10, batch_size=50)

model fitting - RCNN
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 1000)         0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 1000, 300)    34220100    input_2[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_2 (SpatialDro (None, 1000, 300)    0           embedding_2[0][0]                
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 1000, 200)    241200      spatial_dropout1d_2[0][0]        
________________________________________________________________________________________

C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 115s 5ms/step - loss: 0.4073 - acc: 0.8012 - val_loss: 0.2538 - val_acc: 0.8968
Epoch 2/10
25000/25000 [==============================] - 114s 5ms/step - loss: 0.2162 - acc: 0.9146 - val_loss: 0.2323 - val_acc: 0.9074
Epoch 3/10
25000/25000 [==============================] - 115s 5ms/step - loss: 0.1262 - acc: 0.9560 - val_loss: 0.2471 - val_acc: 0.9100
Epoch 4/10
25000/25000 [==============================] - 116s 5ms/step - loss: 0.0643 - acc: 0.9778 - val_loss: 0.2940 - val_acc: 0.9065
Epoch 5/10
25000/25000 [==============================] - 118s 5ms/step - loss: 0.0267 - acc: 0.9914 - val_loss: 0.4523 - val_acc: 0.8890
Epoch 6/10
25000/25000 [==============================] - 115s 5ms/step - loss: 0.0163 - acc: 0.9948 - val_loss: 0.4353 - val_acc: 0.9018
Epoch 7/10
25000/25000 [==============================] - 114s 5ms/step - loss: 0.0112 - acc: 0.9965 - val_loss: 0.5091 - 

In [29]:
# use word2vec_sg 
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_dropout = SpatialDropout1D(0.3)(embedded_sequences)
l_gru = Bidirectional(CuDNNGRU(100, return_sequences=True))(l_dropout)
l_cov = Conv1D(64, kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(l_gru)
avg_pool = GlobalAveragePooling1D()(l_cov)
max_pool = GlobalMaxPooling1D()(l_cov)
l_conc = concatenate([avg_pool, max_pool])
preds = Dense(2, activation='softmax')(l_conc)
model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

print("model fitting - RCNN")
model.summary()
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          nb_epoch=10, batch_size=50)

model fitting - RCNN
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1000)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1000, 300)    34220100    input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_1 (SpatialDro (None, 1000, 300)    0           embedding_1[0][0]                
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 1000, 200)    241200      spatial_dropout1d_1[0][0]        
________________________________________________________________________________________

C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 117s 5ms/step - loss: 0.3737 - acc: 0.8248 - val_loss: 0.2200 - val_acc: 0.9117
Epoch 2/10
25000/25000 [==============================] - 114s 5ms/step - loss: 0.1942 - acc: 0.9246 - val_loss: 0.2245 - val_acc: 0.9100
Epoch 3/10
25000/25000 [==============================] - 114s 5ms/step - loss: 0.1135 - acc: 0.9589 - val_loss: 0.2405 - val_acc: 0.9130
Epoch 4/10
25000/25000 [==============================] - 113s 5ms/step - loss: 0.0556 - acc: 0.9809 - val_loss: 0.2886 - val_acc: 0.9079
Epoch 5/10
25000/25000 [==============================] - 114s 5ms/step - loss: 0.0268 - acc: 0.9914 - val_loss: 0.3773 - val_acc: 0.9025
Epoch 6/10
25000/25000 [==============================] - 114s 5ms/step - loss: 0.0150 - acc: 0.9951 - val_loss: 0.5421 - val_acc: 0.8853
Epoch 7/10
25000/25000 [==============================] - 114s 5ms/step - loss: 0.0120 - acc: 0.9962 - val_loss: 0.5120 - 

In [65]:
# use word2vec
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_dropout = SpatialDropout1D(0.3)(embedded_sequences)
l_gru = Bidirectional(CuDNNGRU(100, return_sequences=True))(l_dropout)
l_cov = Conv1D(64, kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(l_gru)
avg_pool = GlobalAveragePooling1D()(l_cov)
max_pool = GlobalMaxPooling1D()(l_cov)
l_conc = concatenate([avg_pool, max_pool])
preds = Dense(2, activation='softmax')(l_conc)
model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

print("model fitting - RCNN")
model.summary()
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          nb_epoch=10, batch_size=50)

model fitting - RCNN
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1000)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1000, 300)    34220100    input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_1 (SpatialDro (None, 1000, 300)    0           embedding_1[0][0]                
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 1000, 200)    241200      spatial_dropout1d_1[0][0]        
________________________________________________________________________________________

C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 118s 5ms/step - loss: 0.3750 - acc: 0.8310 - val_loss: 0.2563 - val_acc: 0.8972
Epoch 2/10
25000/25000 [==============================] - 115s 5ms/step - loss: 0.2428 - acc: 0.9026 - val_loss: 0.2330 - val_acc: 0.9074
Epoch 3/10
25000/25000 [==============================] - 114s 5ms/step - loss: 0.1878 - acc: 0.9257 - val_loss: 0.2205 - val_acc: 0.9158
Epoch 4/10
25000/25000 [==============================] - 114s 5ms/step - loss: 0.1508 - acc: 0.9424 - val_loss: 0.2494 - val_acc: 0.9093
Epoch 5/10
25000/25000 [==============================] - 113s 5ms/step - loss: 0.1122 - acc: 0.9590 - val_loss: 0.2498 - val_acc: 0.9129
Epoch 6/10
25000/25000 [==============================] - 112s 4ms/step - loss: 0.0776 - acc: 0.9726 - val_loss: 0.2597 - val_acc: 0.9146
Epoch 7/10
25000/25000 [==============================] - 112s 4ms/step - loss: 0.0547 - acc: 0.9811 - val_loss: 0.3037 - 

In [126]:
# use word2vec not including number and punctuation
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_dropout = SpatialDropout1D(0.3)(embedded_sequences)
l_gru = Bidirectional(CuDNNGRU(100, return_sequences=True))(l_dropout)
l_cov = Conv1D(64, kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(l_gru)
avg_pool = GlobalAveragePooling1D()(l_cov)
max_pool = GlobalMaxPooling1D()(l_cov)
l_conc = concatenate([avg_pool, max_pool])
preds = Dense(2, activation='softmax')(l_conc)
model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

print("model fitting - RCNN")
model.summary()
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          nb_epoch=10, batch_size=50)

model fitting - RCNN
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1000)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1000, 300)    34220100    input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_1 (SpatialDro (None, 1000, 300)    0           embedding_1[0][0]                
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 1000, 200)    241200      spatial_dropout1d_1[0][0]        
________________________________________________________________________________________

C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 124s 5ms/step - loss: 0.3790 - acc: 0.8302 - val_loss: 0.2576 - val_acc: 0.8960
Epoch 2/10
25000/25000 [==============================] - 123s 5ms/step - loss: 0.2518 - acc: 0.8986 - val_loss: 0.2553 - val_acc: 0.9010
Epoch 3/10
25000/25000 [==============================] - 123s 5ms/step - loss: 0.1934 - acc: 0.9240 - val_loss: 0.2480 - val_acc: 0.9088
Epoch 4/10
25000/25000 [==============================] - 123s 5ms/step - loss: 0.1464 - acc: 0.9434 - val_loss: 0.2539 - val_acc: 0.9069
Epoch 5/10
25000/25000 [==============================] - 123s 5ms/step - loss: 0.1018 - acc: 0.9631 - val_loss: 0.2719 - val_acc: 0.9073
Epoch 6/10
25000/25000 [==============================] - 122s 5ms/step - loss: 0.0675 - acc: 0.9766 - val_loss: 0.2991 - val_acc: 0.9064
Epoch 7/10
25000/25000 [==============================] - 123s 5ms/step - loss: 0.0424 - acc: 0.9850 - val_loss: 0.3354 - 

In [37]:
# applying a more complex convolutional approach
convs = []
filter_sizes = [3,4,5]

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

for fsz in filter_sizes:
    l_conv = Conv1D(nb_filter=128,filter_length=fsz,activation='relu')(embedded_sequences)
    l_pool = MaxPooling1D(5)(l_conv)
    convs.append(l_pool)
l_merge = Concatenate(axis=1)(convs)    
l_pool1 = GlobalMaxPooling1D()(l_merge)
l_dense = Dense(128, activation='relu')(l_pool1)
preds = Dense(2, activation='softmax')(l_dense)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

print("model fitting - more complex convolutional neural network")
model.summary()
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          nb_epoch=8, batch_size=50)

C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=128, kernel_size=3)`
  if __name__ == '__main__':
C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=128, kernel_size=4)`
  if __name__ == '__main__':
C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=128, kernel_size=5)`
  if __name__ == '__main__':
C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


model fitting - more complex convolutional neural network
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 1000)         0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 1000, 300)    34220100    input_3[0][0]                    
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, 998, 128)     115328      embedding_2[1][0]                
__________________________________________________________________________________________________
conv1d_4 (Conv1D)               (None, 997, 128)     153728      embedding_2[1][0]                
___________________________________________________

In [66]:
# applying a more complex convolutional approach
convs = []
filter_sizes = [3,4,5]

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

for fsz in filter_sizes:
    l_conv = Conv1D(nb_filter=128,filter_length=fsz,activation='relu')(embedded_sequences)
    l_pool = MaxPooling1D(5)(l_conv)
    convs.append(l_pool)
l_merge = Concatenate(axis=1)(convs)    
l_pool1 = GlobalMaxPooling1D()(l_merge)
l_dense = Dense(128, activation='relu')(l_pool1)
preds = Dense(2, activation='softmax')(l_dense)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

print("model fitting - more complex convolutional neural network")
model.summary()
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          nb_epoch=8, batch_size=50)

C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=128, kernel_size=3)`
  if __name__ == '__main__':
C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=128, kernel_size=4)`
  if __name__ == '__main__':
C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=128, kernel_size=5)`
  if __name__ == '__main__':
C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


model fitting - more complex convolutional neural network
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 1000)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1000, 300)    34220100    input_2[0][0]                    
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 998, 128)     115328      embedding_1[1][0]                
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, 997, 128)     153728      embedding_1[1][0]                
___________________________________________________

In [128]:
# applying a more complex convolutional approach
convs = []
filter_sizes = [3,4,5]

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

for fsz in filter_sizes:
    l_conv = Conv1D(nb_filter=128,filter_length=fsz,activation='relu')(embedded_sequences)
    l_pool = MaxPooling1D(5)(l_conv)
    convs.append(l_pool)
l_merge = Concatenate(axis=1)(convs)    
l_pool1 = GlobalMaxPooling1D()(l_merge)
l_dense = Dense(128, activation='relu')(l_pool1)
preds = Dense(2, activation='softmax')(l_dense)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

print("model fitting - more complex convolutional neural network")
model.summary()
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          nb_epoch=8, batch_size=50)

C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=128, kernel_size=3)`
  if __name__ == '__main__':
C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=128, kernel_size=4)`
  if __name__ == '__main__':
C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=128, kernel_size=5)`
  if __name__ == '__main__':
C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


model fitting - more complex convolutional neural network
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 1000)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1000, 300)    34220100    input_3[0][0]                    
__________________________________________________________________________________________________
conv1d_5 (Conv1D)               (None, 998, 128)     115328      embedding_1[2][0]                
__________________________________________________________________________________________________
conv1d_6 (Conv1D)               (None, 997, 128)     153728      embedding_1[2][0]                
___________________________________________________